
# Example of High Gamma Filter

Below is a code sample for extracting high gamma power from a raw data file, followed by permutation cluster stats on that high gamma power data


In [75]:
import ieeg.viz.utils
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.calc import stats, scaling
from ieeg.process import parallelize
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import os
import numpy as np

import pickle

In [77]:
def relabel_axes(old_min, old_max, new_min, new_max):
    scale = (new_max - new_min) / (old_max - old_min)

    def format_func(value, tick_number):
        return new_min + value * scale

    plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))

### grab the data and set up variables

In [76]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)



print(layout.derivatives)
print(layout.derivatives.keys())

# raw = raw_from_layout(layout, subject=sub,
#                         extension='.edf', preload=True)
subjects = layout.get(return_type="id", target="subject")
for sub in subjects:

    if sub == 'D0057':
        continue

    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                            extension='.edf', desc='clean', preload=False) #get line-noise filtered data

    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)


    # do filtering now
    ## Crop raw data to minimize processing time
    new = crop_empty_data(filt, )

    # Mark channel outliers as bad
    # new.info['bads'] = channel_outlier_marker(new, 4)
    # Exclude bad channels
    good = new.copy().drop_channels(filt.info['bads'])
    good.load_data()

    # CAR
    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    # Remove intermediates from mem
    del new

    # make baseline 
    # make stimulus baseline EpochsTFR
    times=[-1, 0.5]

    trials = trial_ieeg(good, "Stimulus", times, preload=True)
    # outliers_to_nan(trials, outliers=10)
    HG_base = gamma.extract(trials, copy=False, n_jobs=1)
    crop_pad(HG_base, "0.5s")

    data_dict = {}  # Create an empty dictionary

    for event, t in zip(("Stimulus", "Response"), ((-1, 1.5), (-1, 1.5))):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, event, times, preload=True)
        # outliers_to_nan(trials, outliers=10)
        HG_ev1 = gamma.extract(trials, copy=False, n_jobs=1)
        crop_pad(HG_ev1, "0.5s")

        HG_ev1_rescaled = rescale(HG_ev1, HG_base, copy=True, mode='ratio') #removed .average() part. Check with Aaron if this is okay.
        # HG_ev1.resample(100)
        # HG_ev1.filenames = good.filenames

        # Store the variable in the dictionary with the desired name
        data_dict[f"HG_ev1_{event}"] = HG_ev1
        data_dict[f"HG_ev1_{event}_rescaled"] = HG_ev1_rescaled
        


    resp_evoke = data_dict["HG_ev1_Response"].average(method=lambda x: np.nanmean(x, axis=0))
    resp_evoke_rescaled = data_dict["HG_ev1_Response_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke = data_dict["HG_ev1_Stimulus"].average(method=lambda x: np.nanmean(x, axis=0))
    stim_evoke_rescaled = data_dict["HG_ev1_Stimulus_rescaled"].average(method=lambda x: np.nanmean(x, axis=0))

    # # # Manually interpolate NaN values
    # resp_evoke.data = np.where(np.isnan(resp_evoke.data), np.interp(resp_evoke.times, resp_evoke.times[~np.isnan(resp_evoke.data[0])], resp_evoke.data[0, ~np.isnan(resp_evoke.data[0])]), resp_evoke.data)

    # # Exclude time points with NaN values
    # valid_time_points = ~np.isnan(resp_evoke.data[0])  # Filter time points without NaNs
    # resp_evoke = resp_evoke.crop(tmin=resp_evoke.times[valid_time_points][0], tmax=resp_evoke.times[valid_time_points][-1])

    # Plot the evoked data
    fig = resp_evoke_rescaled.plot()
    fig.savefig(save_dir + '_HG_ev1_Response_rescaled.png')

    fig = stim_evoke_rescaled.plot()
    fig.savefig(save_dir + '_HG_ev1_Stimulus_rescaled.png')


    # decimate for stats
    resp = data_dict["HG_ev1_Response"] #i think it shouldn't be the rescaled data because that is already divided by baseline, right? So it shouldn't be significantly different than baseline.
    stim = data_dict["HG_ev1_Stimulus"]
    HG_base.decimate(2)
    resp.decimate(2)
    stim.decimate(2)

    # import scipy
    resp_mask = stats.time_perm_cluster(resp._data, HG_base._data, 0.05, axis=0,
                                n_perm=1000, n_jobs=6, ignore_adjacency=1)
    stim_mask = stats.time_perm_cluster(stim._data, HG_base._data, 0.05, axis=0,
                                n_perm=1000, n_jobs=6, ignore_adjacency=1)
    



    # plot stats
    plt.figure(figsize=(16, 12))  # Adjust the width and height as needed
    plt.imshow(stim_mask)
    relabel_axes(0, 2500, -1000, 1500)
    plt.savefig(save_dir + '_stimulus_stats.png', dpi=300)

    plt.figure(figsize=(16, 12))
    plt.imshow(resp_mask)
    relabel_axes(0, 2500, -1000, 1500)
    plt.savefig(save_dir + '_response_stats.png', dpi=300)


{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 6 | Sessions: 0 | Runs: 24})
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']


FileNotFoundError: No files match your search terms